# Cluster simulations with new uncertainties

### Modules

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from itertools import product

from Extraction.pre_processing import cluster_df_list
from Extraction.Classfile import star_cluster

from My_tools import my_utility

%matplotlib notebook

### Paths

In [2]:
output_path = my_utility.set_output_path(main_path="/Users/alena/Library/CloudStorage/OneDrive-Personal/Work/PhD/Projects/Isochrone_Archive/Coding_logs/")

mastertable_path = "/Users/alena/PycharmProjects/PaperI/data/Isochrones/Mastertable_Archive.csv"

## 1. Define intervals of uncertainty

In [3]:
f_plx = [0.01, 0.1]
f_bin = [0.2, 0.5]
f_cont = [0.05, 0.9]
extinct = [0.1,0.789]

## 2. Grid of the ranges

In [4]:
combinations = np.array(list(product(f_plx, f_bin, extinct, f_cont)))

## 3. Take representative isochrones (young middle and old ages)

In [5]:
# define clusters
clusters = ["delta Sco", "Melotte_22", "NGC_2632"]

### 3.1. Get mean cluster distances

Use the mean distances as approximation for the distance of the synthetic stars generated along the empirical isochrones.

In [6]:
# load isochrone table
mastertable = pd.read_csv(mastertable_path)
filtered_df = mastertable[mastertable["Cluster"].isin(clusters)]

# Define cluster objects (for est. distances)
Archive_df = pd.concat(cluster_df_list, axis=0)

distances = []
for cluster in clusters:
    OC = star_cluster(cluster, Archive_df)
    distances.append(np.mean(OC.distance))

#### Plot the isochrone points

In [17]:
#sns.set_style("darkgrid")
plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["font.family"] = "STIXGeneral"
plt.rcParams["font.size"] = 15

# design colorbar and norm
a = plt.get_cmap("YlGnBu_r")
norm = plt.Normalize(filtered_df.ref_age.min(), filtered_df.ref_age.max())
sm = plt.cm.ScalarMappable(cmap="YlGnBu_r", norm=norm)
sm.set_array([])

# age limits if necessary
age_low = 6
age_up = 11

# Poster figure (single panel)
fig_poster, ax = plt.subplots(1, 1, figsize=(3.5, 4.4))
plt.subplots_adjust(left=0.16, bottom=0.11, top=0.98, right=0.97, wspace=0.0)

BPRP = sns.lineplot(data=filtered_df, x="BPRP_isochrone_x", sort = False, estimator = None, units = "Cluster",
                    y="BPRP_isochrone_y", hue="ref_age", palette=a, hue_norm=norm, legend=False,
                    lw=1, ax=ax).set(
    xlabel=r"$\mathrm{G}_{\mathrm{BP}} - \mathrm{G}_{\mathrm{RP}}$", ylabel=r"$\mathrm{M}_{\mathrm{G}}$")
ax.set_xlim(-0.5, 4.1)
ax.set_ylim(14, -3.5)
ax.set_xlabel(r"$\mathrm{G}_{\mathrm{BP}} - \mathrm{G}_{\mathrm{RP}}$",labelpad=1)
ax.set_ylabel(r"$\mathrm{M}_{\mathrm{G}}$", labelpad = 1)
c = fig_poster.colorbar(sm, ax=ax, location='right', fraction=0.15, aspect=20, pad=0.05)
cax = c.ax
plt.text(5.2, 18, 'log age')
plt.grid(True)
plt.tick_params(axis='both')

fig_poster.show()


<IPython.core.display.Javascript object>

#### Function for calculating apparent G

In [8]:
def apparent_G(M, dist):
    d = np.empty(shape = len(M))
    d.fill(dist)
    return 5*np.log10(d) - 5 + M

## 4. Add uncertainties

*Note:* Trial run for 1 random combo and 1 cluster

In [9]:
# Choose a random row
random_row = combinations[np.random.randint(0, combinations.shape[0])]
u_plx, binarity, field_cont, ext = random_row

# choose cluster
c_id = 1
cluster_data = filtered_df[filtered_df["Cluster"]==clusters[c_id]]
abs_G = cluster_data["BPRP_isochrone_y"]
bp_rp = cluster_data["BPRP_isochrone_x"]

# calculate apparent G mag
green = apparent_G(abs_G, distances[c_id])

### 4.1. Parallax uncertainty

In [10]:
lower_bound = -u_plx
upper_bound = u_plx
num_samples = len(green)

# Calculate mean and standard deviation based on the desired bounds
mean = (upper_bound + lower_bound) / 2
std_dev = (upper_bound - lower_bound) / 6  # Dividing by 6 for 99.7% coverage within the range

# Generate a normal distribution
normal_distribution = np.random.normal(mean, std_dev, num_samples)

# convert mean distance to plx
plx = 1000/distances[c_id]
new_dist = 1000/ (plx + plx * normal_distribution)

print(distances[c_id], np.mean(new_dist), np.std(new_dist), max(new_dist), min(new_dist))

M_g_new = green - 5 * np.log10(new_dist)+5

# uncertainty_dist = np.random.uniform(-u_plx, u_plx,len(green))

136.2847983937677 136.49191495479127 4.702667415439612 154.18140648617097 124.24133188604343


### 3.2. Binary fraction

A fraction of the stars is shifted by -0.753 mag

In [11]:
binary_frame = M_g_new.copy()
# Randomly sample 30% of the elements
sampled_indices = binary_frame.sample(frac=binarity).index

# Apply the shift to the original series based on sampled indices
binary_frame.loc[sampled_indices] -= 0.753

### 3.3. Extinction

In [12]:
# make into dataframe
cluster_df = pd.DataFrame(data = np.stack([bp_rp, binary_frame], axis = 1), columns=["bp_rp", "M_g"])

In [13]:
A_BP = 1.212 * ext
A_RP = 0.76 * ext
E = A_BP - A_RP

cluster_df["M_g"] += ext
cluster_df["bp_rp"] += E

### 3.4. Contamination data

*Note:* Function does not have to be called again (from SigMA)

In [14]:
# load slimmed catalog
file_path = '/Users/alena/PycharmProjects/PaperI/data/Gaia_DR3/Gaia_DR3_500pc_1percent.csv'
data = pd.read_csv(file_path)

# Randomly sample contaminated entries
n = int(len(green)*field_cont)
sampled_data = data.sample(n=n)

# use only the relevant columns
field_df = sampled_data[["parallax", "parallax_error","ruwe", "phot_g_mean_mag", "phot_bp_mean_mag", "phot_rp_mean_mag"]]

# generate CMD columns
field_df.loc[:,"bp_rp"] = field_df.loc[:,'phot_bp_mean_mag'] - field_df.loc[:,'phot_rp_mean_mag']
field_df.loc[:,"M_g"] = field_df.loc[:,'phot_g_mean_mag'] - 5*np.log10(1000/ field_df.loc[:, "parallax"]) + 5

/var/folders/z6/y4rdkpdx3vlbwtmngx04x_1w0000gp/T/ipykernel_2088/587093552.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
# merge with other df
common_columns = cluster_df.columns.intersection(field_df.columns)

# Concatenate based on common columns and reindex
OC_df = pd.concat([cluster_df, field_df[common_columns]], axis=0)

In [16]:
fig, ax = plt.subplots(2,3, figsize = (8,6))
plt.subplots_adjust(top = 0.95, left = 0.05, right= 0.98, hspace=0.2, wspace=0.15, bottom =0.05)
axes = ax.ravel()

 #### Plot original
axes[0].scatter(bp_rp, abs_G, s=5, color = "blue", label="original")
axes[0].set_ylim(15, -4)
axes[0].legend(loc="best")

# plx uncertainty
axes[1].scatter(bp_rp, M_g_new, s = 5, color = "orange", label = "plx uncertainty")
axes[1].set_ylim(15,-4)
axes[1].legend(loc="best")
axes[1].set_title(u_plx)

# binarity
axes[2].scatter(bp_rp, binary_frame, s = 5, color = "red", label = "binaries")
axes[2].set_ylim(15,-4)
axes[2].legend(loc="best")
axes[2].set_title(binarity)

# extinction
axes[3].scatter(cluster_df["bp_rp"], cluster_df["M_g"], s = 5, color = "green", label = "extinction")
axes[3].set_ylim(15,-4)
axes[3].legend(loc="best")
axes[3].set_title(ext)

# contamination
axes[4].scatter(OC_df.bp_rp, OC_df.M_g, s = 5, color = "violet", label = "field")
axes[4].set_ylim(15,-4)
axes[4].legend(loc="best")
axes[4].set_title(field_cont)

<IPython.core.display.Javascript object>

Text(0.5, 1.0, '0.9')